# Generate word2vec model - include all new topic words

In [1]:
from gensim.models import Word2Vec, KeyedVectors
import gensim
import time
import numpy as np
from tqdm import tqdm
from pprint import pprint
import pickle
import spacy
import re
import operator
import spacy_udpipe

In [2]:
nlp = spacy.load('it_core_news_sm')

In [3]:
# set variables 
LANG = 'it'
MAX_WORDS = 500000  # maximum processed words to small model 

MODEL_DIR = '/mnt/data/data/models-fasttext/'
TOPICS_DIR = '/home/dzon/kajo/topics/topic_lists-based_on_en-US/'

model_filename = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2.vec.bin'

excluded_model_name = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2_excluded.vec.bin'

model_orig_filename = MODEL_DIR + 'cc.'+ LANG +'.300.vec.bin'
model_orig_filename_vec = MODEL_DIR + 'cc.'+ LANG +'.300.vec.gz'

pickled_words_lemmas = MODEL_DIR + LANG + '_ft_all_words_lemmas.pickle'

lemma_list_file = TOPICS_DIR + 'topics_lemma_'+ LANG +'.lst'
nolemma_list_file = TOPICS_DIR + 'topics_'+ LANG +'.lst'

excluded_filename = TOPICS_DIR + 'topics_' + LANG + '-not_in_model.lst'

lemma_list_file_excluded = TOPICS_DIR + 'topics_lemma_excluded_' + LANG + '.lst'
nolemma_list_file_excluded = TOPICS_DIR + 'topics_excluded_' + LANG + '.lst'


In [4]:
# load the big model
model_big = KeyedVectors.load_word2vec_format(model_orig_filename, binary=True)

In [5]:
#load pickled lemma list
with open(pickled_words_lemmas, 'rb') as f:
    all_words_lemmas = pickle.load(f)
pprint(all_words_lemmas[:50])

[('di', 'di'),
 ('e', 'e'),
 ('il', 'il'),
 ('la', 'la'),
 ('in', 'in'),
 ('che', 'che'),
 ('a', 'a'),
 ('per', 'per'),
 ('un', 'un'),
 ('del', 'del'),
 ('è', 'essere'),
 ('con', 'con'),
 ('della', 'della'),
 ('da', 'da'),
 ('una', 'una'),
 ('i', 'il'),
 ('non', 'non'),
 ('le', 'la'),
 ('si', 'si'),
 ('al', 'al'),
 ('nel', 'nel'),
 ('dei', 'dio'),
 ('sono', 'essere'),
 ('più', 'molto'),
 ('Il', 'il'),
 ('alla', 'alla'),
 ('come', 'come'),
 ('La', 'la'),
 ('ha', 'avere'),
 ('delle', 'della'),
 ('anche', 'anche'),
 ('o', 'o'),
 ('ma', 'ma'),
 ('l', 'l'),
 ('gli', 'il'),
 ('nella', 'nella'),
 ('dal', 'dal'),
 ('su', 'su'),
 ('ed', 'ed'),
 ('ad', 'ad'),
 ('lo', 'il'),
 ('questo', 'questo'),
 ('se', 'se'),
 ('essere', 'essere'),
 ('cui', 'cui'),
 ('_', '_'),
 ('tra', 'tra'),
 ('due', 'due'),
 ('dalla', 'dalla'),
 ('I', 'i')]


In [6]:
# generate helpful dictionaries/lists
big_lemmas_set = set([l for (_,l) in all_words_lemmas])
print(len(big_lemmas_set))

big_words_set = set([w for (w,_) in all_words_lemmas])
print(len(big_words_set))

lemmas = {}
words_lower_orig = {}
big_lemma_word_dic = {}
big_word_lemma_dic = {}
for (w,l) in all_words_lemmas:
    big_word_lemma_dic[w] = l
    
    #dictionary of lowercase word -> all original words
    if w.lower() in words_lower_orig:
        words_lower_orig[w.lower()].append(w)
    else:
        words_lower_orig[w.lower()] = [w]
        
    if l in big_lemma_word_dic:
        big_lemma_word_dic[l].append(w)
        lemmas[l] += 1
    else:
        big_lemma_word_dic[l] = [w]
        lemmas[l] = 1
        
# load all the words from big model
words = model_big.wv.index2word
words_lower = [w.lower() for w in words]

#words_lemma = model_lemma_big.index2word       

1144222
1527086


/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [7]:
# sort the lemmas from most frequent
sorted_lemmas = sorted(lemmas.items(), key=operator.itemgetter(1), reverse=True)
# trying first words
sorted_lemmas[:20]

[('avere', 58),
 ('potere', 56),
 ('dire', 55),
 ('dovere', 54),
 ('vedere', 53),
 ('essere', 52),
 ('fare', 52),
 ('venire', 51),
 ('volere', 51),
 ('scrivere', 50),
 ('sapere', 50),
 ('mettere', 50),
 ('credere', 49),
 ('uscire', 49),
 ('tenere', 49),
 ('dare', 48),
 ('trovare', 48),
 ('andare', 48),
 ('vivere', 48),
 ('guardare', 48)]

In [8]:
# create vectors for the small models - average vectors for lemmas
# process only first MAX_WORDS in the big model
tmp_words = [None] * len(model_big.wv.index2word)
tmp_vecs = [None] * len(model_big.wv.index2word) 
for i,(word,lemma) in enumerate(all_words_lemmas[:MAX_WORDS]):
    lemma_words_list = big_lemma_word_dic[lemma]
    # add only lemmas made from at least 2 words
    if len(lemma_words_list)<2:
        continue
    tmp_vectors = []
    tmp_indexes = []
    error_with_word = False
    for word2 in lemma_words_list:
        try:
            tmp_vectors.append(model_big[word2])  
            tmp_indexes.append(model_big.vocab[word2].index)
        except:
            print(i, "error with ", word, " lemma ", lemma, "words", lemma_words_list)
            error_with_word = True
            continue
    if not error_with_word:
        try:
            tmp_words[min(tmp_indexes)] = lemma
            tmp_vecs[min(tmp_indexes)] = list(np.average(tmp_vectors, axis=0))
        except:
            print(i, "error with lemma ", lemma, " - words", lemma_words_list, " indexes: ", tmp_indexes)
            # note - we are processing first MAX_WORDS from the model after the filter out of the "non-words"
            # so some of the last words in that range may refer to words on bigger index than MAX_WORDS

# filter the lists (some positions remained None)
final_words = [i for i in tmp_words if i] 
final_vecs = [i for i in tmp_vecs if i != None] 

print(len(final_vecs), len(final_words))
len(final_vecs[0])  

del tmp_vecs
del tmp_words

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


161009 161009


In [9]:
# create new gensim word2vec model with the vectors:
new_vector_space = gensim.models.keyedvectors.Word2VecKeyedVectors(300)
new_vector_space.add(final_words,final_vecs)

In [10]:
# test the model
pprint(new_vector_space.most_similar('padre'))
pprint(new_vector_space.most_similar_cosmul(positive=['donna', 'monarca'], negative=['uomo']))

[('figlio', 0.7492443323135376),
 ('figliolo', 0.6482381820678711),
 ('padri', 0.6285314559936523),
 ('papà', 0.6097040176391602),
 ('dio', 0.6067836284637451),
 ('nipote', 0.5925706624984741),
 ('madre', 0.5916823744773865),
 ('figlia', 0.5908894538879395),
 ('nonno', 0.5874237418174744),
 ('signore', 0.5843079090118408)]
[('regina', 0.8882865309715271),
 ('monarchia', 0.8768400549888611),
 ('principessa', 0.863275945186615),
 ('monarchico', 0.8495009541511536),
 ('concubina', 0.8462414741516113),
 ('viceregina', 0.8438517451286316),
 ('sovrano', 0.8428009748458862),
 ('principessina', 0.8351831436157227),
 ('zarina', 0.832023024559021),
 ('nobildonna', 0.8302243947982788)]


In [11]:
# save the model
new_vector_space.save_word2vec_format(model_filename, binary=True)

In [12]:
#model = KeyedVectors.load_word2vec_format(model_filename, binary=True)
model = new_vector_space
words_model = model.wv.index2word

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# load topic listst
with open(nolemma_list_file) as fp:
    topics_nolemma = fp.readlines() 
print(len(topics_nolemma))
    
# load the lemma list
# with open(lemma_list_file) as fp:
#    topics = fp.readlines()
# print(len(topics))

topics = []
# generate the lemma list
for topic in tqdm(topics_nolemma):
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []
        
    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    if len(lemmas) == 0:
        print('lemmas = 0 ', topic)
    topics.append("_".join(lemmas))
    
print(len(topics))    
    
#write the lemma topic list
with open(lemma_list_file, 'wt', encoding='utf8') as f:
    for topic_lemma in topics:
        f.write(topic_lemma + "\n")    

  0%|          | 18/5308 [00:00<00:30, 171.77it/s]

5308


  2%|▏         | 84/5308 [00:00<00:26, 198.27it/s]

lemmas = 0  forza lavoro 


 21%|██▏       | 1136/5308 [00:05<00:18, 225.05it/s]

...

lemmas = 0  tempo 


 91%|█████████ | 4805/5308 [00:22<00:02, 217.53it/s]

lemmas = 0  registrazione 


100%|██████████| 5308/5308 [00:24<00:00, 218.01it/s]

5308


In [14]:
# some words are wrongly identified as STOP words - like 'stato'='state' is identified as "go"
doc = nlp('stato')
for t in doc:
    print(t.lemma_, t.is_stop)

stare True


In [15]:
print(len(topics),len(topics_nolemma))  

5308 5308


In [16]:
#try if the topic-lemma words exist in the models  

words_to_manual_add = {}

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic = topic.strip()
    topic_words = topic.split('_')
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    for word, word_nolemma in zip(topic_words, nolemmas):
        #print(word)
        found = 0
        if word in words_model:
            found = 1
            #print("l1")
        elif word in big_lemmas_set:
            found = 1
            print("l2", word)
        elif word in words_lower:
            found = 1
            print("l3", word)
        # nolemma words are in PL case probably not needed and all lemmas exist in lemma list
        elif word_nolemma in words_model:
            found = 1
            print("w1", word)
        elif word_nolemma in big_lemmas_set:
            found = 1
            print("w2", word)
        elif word_nolemma in words_lower:
            found = 1
            print("w3", word)

        if found == 0:
            print(i, 'needs to be added manually: ', word, " - ", word_nolemma, " - ", topic_nolemma.strip())
            if word not in words_to_manual_add:
                words_to_manual_add[word] = topic_nolemma.strip()
                    
print(len(words_to_manual_add))          

!!!!! error - not equal length:  [] ['']
l2 salinizzazione
l2 salinizzazione
l2 dissodamento
l2 molluschicoltura
l2 abiotico
l2 aridocoltura
l2 microclimatico
l2 biogeografiche
l2 riutilizzazione
l2 desalinazione
l2 ostricoltura
746 needs to be added manually:  leislazione  -  leislazione  -  leislazione fiscale
769 needs to be added manually:  lotiche  -  lotiche  -  acque lotiche
l2 effluente
l2 pianificatoria
l2 ripariale
l2 parcare
l2 trofico
l2 geomorfici
l2 effluente
!!!!! error - not equal length:  [] ['']
l2 parcare
l2 parcare
l2 regolativa
l2 vegetazionale
l2 parcare
1287 needs to be added manually:  ecocittadinanza  -  ecocittadinanza  -  ecocittadinanza
l2 idrogramma
l2 thermoselect
l2 caducifoglie
l2 ornitiche
!!!!! error - not equal length:  [] ['']
l2 parcare
l2 ambientalmente
l3 co2
l2 bentico
l2 parcare
l2 desalinazione
l2 parcare
l2 microcentrale
l2 aox
l2 foraggiera
l2 semiaridi
l2 ecoaudit
2018 needs to be added manually:  medidionale  -  medidionale  -  Asia medidio

In [17]:
# for manual correcting of words
pprint(words_to_manual_add)

{'algicida': 'algicida',
 'aliciclico': 'composto aliciclico',
 'antiincrostante': 'agente antiincrostante',
 'antropogeni': 'fattori antropogeni',
 'attenuativa': 'misura attenuativa',
 'bioaccumulativo': 'inquinante bioaccumulativo',
 'bioconcentrazione': 'fattore di bioconcentrazione',
 'carcinogenità': 'test sulla carcinogenità',
 'chemiadsorbimento': 'chemiadsorbimento',
 'cloroetilene': 'cloroetilene',
 'clorofluorurati': 'idrocarburi clorofluorurati parzialmente alogenati',
 'conimetro': 'conimetro ad urto',
 'crisofite': 'crisofite',
 'disinchiostratura': 'disinchiostratura',
 'ecocittadinanza': 'ecocittadinanza',
 'ecomarketing': 'ecomarketing',
 'ecopediatria': 'ecopediatria',
 'edafologia': 'edafologia',
 'eluzione': 'eluzione',
 'emulsificazione': 'emulsificazione',
 'fregolatoio': 'fregolatoio',
 'geogenico': 'fattore geogenico',
 'geotecnologia': 'geotecnologia',
 'halons': 'messa al bando dei clorofluorocarburi e degli halons',
 'idrometeorologia': 'idrometeorologia',
 '

In [18]:
# the lemmas not in model were manually checked if they exist in other models in some variation
word = "leislazione"
print(word)
try:
    model[word]
    print('model')
    pprint(model.most_similar(word))
except:
    pass

if word in big_lemmas_set:
    print('model_lemma_big', big_lemma_word_dic[word])

try:
    model_big[word]
    print('model big')
    pprint(model_big.most_similar(word))
except:
    pass

# if the original word with uppercase
if word in words_lower_orig:
    print('model big LOWER', words_lower_orig[word])
    pprint(model_big.most_similar(words_lower_orig[word]))
    

leislazione


In [19]:
# for cases the lemmas aren't correct or don't exist in the dic
# manually made dictionary of "lemma" : "word which exists in some model" or "word1;word2" for the average
# e.g. 'mikroekosystém': 'mikro;ekosystém', 'bahný': 'bahno'
new_words_manually_checked = {}
    
len(new_words_manually_checked)

0

In [20]:
# function which finds the word in all the available models and returns it's vector (average vector)
def get_vector(word):
    # word is in small model
    try:
        return model[word]
    except:
        pass
    
    # word is in lemmas from big model - return average if the lemma was result of more words
    if word in big_lemmas_set:
        lemma_words_list = big_lemma_word_dic[word]
        tmp_vectors = []
        for word2 in lemma_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

    try:
        return model_big[word]
    except:
        pass
    
    # if the original word with uppercase
    if word in words_lower_orig:
        orig_words_list = words_lower_orig[word]
        tmp_vectors = []
        for word2 in orig_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

        #return model_big[words_lower_orig[word]]
    
    return []

In [21]:
# check if the word vectors are the same
#get_vector('Antarktyki') == get_vector('antarktyki')


In [22]:
# ading words from manually checked list
new_vectors = []
new_words = []
for i, word in enumerate(new_words_manually_checked.keys()):
    
    tmp_vectors = []
    print(i, word)
    for tmp_word in new_words_manually_checked[word].split(';'):
        vector = get_vector(tmp_word)
        if vector != []:
            tmp_vectors.append(vector)
        else:
            print("XXX - some ERROR, word doesn't have vector, cannot add: ", tmp_word)
    if word not in new_words:
        new_vectors.append(np.average(tmp_vectors, axis=0))
        new_words.append(word)
print(len(new_vectors),len(new_words))        

0 0


In [23]:
# add new words to the model
if len(new_words) > 0:
    model.add(new_words,new_vectors)

In [24]:
print(len(topics),len(topics_nolemma))  

5308 5308


In [25]:
# 2nd run - checking if there are still unavailable words in topics
excluded_topics = []
#try if the topic-lemma words exist in the model   

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma_orig = topic_nolemma.strip()
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        #mostly because the lemma topic is empty ""
        excluded_topics.append(topic_nolemma_orig)
        continue
        
    found = 1
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        #print(word)
       
        vector = get_vector(word)
        if vector == []:
            print('nonexistend word: ', word, " - ", topic_nolemma)
            vector2 = get_vector(word_nolemma)
            if vector2 == []:
                found = 0
                print('nonexistend word: ', word_nolemma, " - ", topic_nolemma)
            else:
                print("Exist nolemma")

    if found == 0:
        print('!!! topic needs to be excluded: ', topic_nolemma_orig)
        if topic_nolemma.strip() not in excluded_topics:
            excluded_topics.append(topic_nolemma_orig)
                    
print(len(excluded_topics)) 

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


!!!!! error - not equal length:  [] ['']
nonexistend word:  leislazione  -  leislazione fiscale 
nonexistend word:  leislazione  -  leislazione fiscale 
!!! topic needs to be excluded:  leislazione fiscale
nonexistend word:  lotiche  -  acque lotiche 
nonexistend word:  lotiche  -  acque lotiche 
!!! topic needs to be excluded:  acque lotiche
!!!!! error - not equal length:  [] ['']
nonexistend word:  ecocittadinanza  -  ecocittadinanza 
nonexistend word:  ecocittadinanza  -  ecocittadinanza 
!!! topic needs to be excluded:  ecocittadinanza
!!!!! error - not equal length:  [] ['']
nonexistend word:  medidionale  -  Asia medidionale 
nonexistend word:  medidionale  -  Asia medidionale 
!!! topic needs to be excluded:  Asia medidionale
nonexistend word:  macrofita  -  macrofita 
nonexistend word:  macrofita  -  macrofita 
!!! topic needs to be excluded:  macrofita
nonexistend word:  eluzione  -  eluzione 
nonexistend word:  eluzione  -  eluzione 
!!! topic needs to be excluded:  eluzione

In [26]:
# save topics containing words which are not present neither model - to be excluded from the topic list
print('excluded topics: ', len(excluded_topics))
pprint(excluded_topics)
with open(excluded_filename, 'wt', encoding='utf8') as f:
    for topic in excluded_topics:
        f.write(topic + "\n")

excluded topics:  57
['forza lavoro',
 'leislazione fiscale',
 'acque lotiche',
 'lavoro',
 'ecocittadinanza',
 'tempo',
 'Asia medidionale',
 'macrofita',
 'eluzione',
 'misura attenuativa',
 'migrazione stagionle',
 'fattori antropogeni',
 'mondo',
 'composto organoclorurato',
 'periphyton',
 'posto di lavoro',
 'stato',
 'composto aliciclico',
 'composti organosiliconici',
 'lievito (saccaromicete)',
 'inquinante bioaccumulativo',
 'chemiadsorbimento',
 'tempo',
 'microclimatologia',
 'nave cisterna a interponte',
 'idrometeorologia',
 'emulsificazione',
 'pedosfera',
 'fregolatoio',
 'fattore geogenico',
 'fattore di bioconcentrazione',
 'messa al bando dei clorofluorocarburi e degli halons',
 'indice saprobico',
 'registrazione',
 'edafologia',
 'radioelemento',
 'agente antiincrostante',
 'sostanza xenobiotica',
 'conimetro ad urto',
 'idrocarburi clorofluorurati parzialmente alogenati',
 'test sulla carcinogenità',
 'ecopediatria',
 'policlorodibenzo-p-diossina',
 'saprobio',
 '

In [27]:
# create new lists which don't containt topics with non-existent words

# load list with excluded topics
with open(excluded_filename, 'rt', encoding='utf8') as fp:
    tmp_topics = fp.readlines()
excluded_set = set()
for topic in tmp_topics:
    excluded_set.add(topic.strip())

with open(nolemma_list_file, 'rt', encoding='utf8') as fp:
        topics_file = fp.readlines()

topics_final = []
topics_lemma = []

excluded = 0
for topic in topics_file:
    topic_orig = topic.strip()
    if topic_orig in excluded_set:
        print("=== EXCLUDE ", topic_orig)
        excluded += 1
        continue
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []

    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics_lemma.append("_".join(lemmas))
    topics_final.append(topic_orig)

    print(topic_orig, ' --- ', "_".join(lemmas))

print('Excluded: ', excluded)
print('Final topics: ', len(topics_lemma), len(topics_final))
    
with open(lemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_lemma in topics_lemma:
        f.write(topic_lemma + "\n")
with open(nolemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_final in topics_final:
        f.write(topic_final + "\n")  

sviluppo locale  ---  sviluppare_locale
sviluppo rurale  ---  sviluppare_rurale
gestione delle risorse idriche  ---  gestione_risorsa_idrico
grado d'istruzione  ---  grado_d_istruzione
istruzione generale  ---  istruzione
argine  ---  argine
diritto comunitario  ---  diritto_comunitario
patrimonio culturale  ---  patrimonio_culturale
terreno agricolo  ---  terreno_agricolo
paesaggio coltivato  ---  paesaggio_coltivare
produzione agricola  ---  produzione_agricolo
sviluppo economico  ---  sviluppare_economico
catena alimentare  ---  catena_alimentare
industria alimentare  ---  industriare_alimentare
qualità degli alimenti  ---  qualità_alimento
riscaldamento globale dell'atmosfera  ---  riscaldamento_globale_atmosfera
effetto serra  ---  effetto_serrare
sviluppo industriale  ---  sviluppare_industriale
mercato del lavoro  ---  mercato
accesso al territorio  ---  accesso_territorio
accorpamento parcellare  ---  accorpamento_parcellare
pianificazione territoriale  ---  pianificazione_terr

credito sulla fiducia  ---  credito_fiducia
inquinamento del suolo  ---  inquinamento_suolare
inquinamento dell'acqua  ---  inquinamento_acqua
tenuta agricola  ---  tenere_agricolo
nuova comunità  ---  nuovo_comunità
economia  ---  economia
conquista di territorio  ---  conquistare_territorio
immagazzinamento di derrate alimentari  ---  immagazzinamento_derrata_alimentario
nutrizione degli animali  ---  nutrizione_animale
produzione forestale  ---  produzione_forestale
pianificazione agricola  ---  pianificazione_agricolo
coltivazione di verdure  ---  coltivazione_verdura
sviluppo lungo la costa  ---  sviluppare_costare
danno al suolo  ---  dare_suolare
politica sociale  ---  politico_sociale
manutenzione dell'ambiente  ---  manutenzione_ambiente
animali  ---  animale
sviluppo culturale  ---  sviluppare_culturale
paesaggio  ---  paesaggio
legislazione agricola  ---  legislazione_agricolo
cura della salute  ---  curare_salute
espansione urbana incontrollata  ---  espansione_urbano_incon

pianificazione  ---  pianificazione
nuove tecnologie  ---  nuovo_tecnologia
trasporto su strada  ---  trasportare_strada
modello di sviluppo (economia)  ---  modellare_sviluppare_economia
trasporto merci  ---  trasportare_merce
legge quadro  ---  leggere_quadrare
politica economica  ---  politico_economico
sistema sociale  ---  sistemare_sociale
uso del suolo  ---  usare_suolare
materiale innovativo  ---  materiale_innovativo
industria del vetro  ---  industriare_vetro
politica della ricerca  ---  politico_ricercare
area in via di sviluppo  ---  area_sviluppare
rendimento (agricoltura)  ---  rendimento_agricoltura
gestione dei rifiuti  ---  gestione_rifiuto
pesca interna  ---  pescare_interno
industria meccanica  ---  industriare_meccanico
inquinante alimentare  ---  inquinare_alimentare
industria del legno  ---  industriare_legnare
integrazione delle politiche gestionali  ---  integrazione_politico_gestionale
trasporto di derrate alimentari  ---  trasportare_derrata_alimentario
indust

strada di accesso  ---  strada_accesso
diritto privato  ---  diritto_privare
diritto pubblico  ---  diritto_pubblicare
chimica del suolo  ---  chimico_suolare
scienze fisiche  ---  scienza_fisico
scienze dell'acqua  ---  scienza_acqua
densità della popolazione  ---  densità_popolazione
zona industriale abbandonata  ---  zona_industriale_abbandonato
politica di governo  ---  politico
cooperazione per lo sviluppo  ---  cooperazione_sviluppare
società (legge)  ---  società_leggere
struttura industriale  ---  strutturare_industriale
struttura economica  ---  strutturare_economico
contratto pubblico  ---  contrattare_pubblicare
persona anziana  ---  persona_anziano
inquinamento della costa  ---  inquinamento_costare
inquinamento delle acque dolci  ---  inquinamento_acqua_dolce
allevamento di pollame  ---  allevamento_pollame
potenzialità del suolo  ---  potenzialità_suolare
mercato dell'ambiente  ---  mercato_ambiente
pompa dell'acqua  ---  pompare_acqua
mercato dell'energia  ---  mercato_e

mammifero terrestre  ---  mammifero_terrestre
disposizioni della CE  ---  disposizione_ce
internet  ---  internet
regolamento legislativo  ---  regolamentare_legislativo
area vulcanica  ---  area_vulcanico
diritto costituzionale  ---  diritto_costituzionale
uccello da preda  ---  uccellare_predare
indice ambientale  ---  indire_ambientale
politica dell'impresa  ---  politico_imprendere
monitoraggio del mare  ---  monitoraggio_mare
formazione continua  ---  formazione_continuo
comunità urbana  ---  comunità_urbano
decisione comunitaria  ---  decisione_comunitario
politica di conservazione  ---  politico_conservazione
rifiuto industriale  ---  rifiutare_industriale
agricoltura industriale  ---  agricoltura_industriale
=== EXCLUDE  acque lotiche
effluente agricolo  ---  effluente_agricolo
ecologia forestale  ---  ecologia_forestale
legislazione sulla concorrenza  ---  legislazione_concorrenza
risorse animali  ---  risorsa_animale
catasto dei terreni  ---  catasto_terreno
medicina del lavo

...

pubblicità  ---  pubblicità
impermeabilizzazione del fondo delle discariche  ---  impermeabilizzazione_fondere_discarica
sponsorizzazione  ---  sponsorizzazione
scarico nell'oceano  ---  scaricare_oceano
cane  ---  cane
clorurazione  ---  clorurazione
batteri fecali  ---  batterio_fecale
interessi  ---  interesse
obbligo di informazione  ---  obbligare_informazione
vulcanismo  ---  vulcanismo
discarica non controllata  ---  discaricare_controllato
inertizzazione  ---  inertizzazione
acido diluito  ---  acido_diluire
procedura delle udienze  ---  procedura_udienza
concentrazione massima ammissibile  ---  concentrazione_massimo_ammissibile
ruscello  ---  ruscello
installazione soggetta ad approvazione  ---  installazione_soggetto_approvazione
centrale nucleare  ---  centrale_nucleare
confronto di prodotti  ---  confrontare_produrre
identificazione degli inquinanti  ---  identificazione_inquinare
rifiuto pastoso  ---  rifiutare_pastoso
fonte d'informazione  ---  fonte_d_informazione
tossi

sentenza  ---  sentenza
reattore nucleare  ---  reattore_nucleare
discarica per rifiuti pericolosi  ---  discaricare_rifiuto_pericoloso
principio del motivo di preoccupazione  ---  principiare_motivare_preoccupazione
responsabilità per danni nucleari  ---  responsabilità_danno_nucleare
asbestosi  ---  asbestosi
guardia costiera  ---  guardia_costiero
=== EXCLUDE  fregolatoio
alcoli  ---  alcoli
agente sbiancante  ---  agire_sbiancare
=== EXCLUDE  fattore geogenico
sale  ---  salire
dizionario  ---  dizionario
gassificazione del carbone  ---  gassificazione_carbone
ferraglia  ---  ferraglia
imposta  ---  impostare
digestore  ---  digestore
salmonella  ---  salmonella
vanadio  ---  vanadio
suono  ---  suonare
pesca di deriva  ---  pescare_derivare
classificazione  ---  classificazione
antibiotico  ---  antibiotico
flottazione  ---  flottazione
leucemia  ---  leucemia
limite ammissibile di esposizione  ---  limite_ammissibile_esposizione
agente olio-assorbente  ---  agire_oliare_assorbire

coagulazione  ---  coagulazione
diossine  ---  diossina
imbibizione  ---  imbibizione
virus  ---  virus
attributo  ---  attributo
adesivo  ---  adesivo
comunicato stampa  ---  comunicare_stampare
felidi  ---  felidi
polietilentereftalato  ---  polietilentereftalato
latitudine  ---  latitudine
previsione delle immissioni  ---  previsione_immissione
tracciamento  ---  tracciamento
danno da manovre militari  ---  dare_manovra_militare
centralizzazione  ---  centralizzazione
permesso  ---  permettere
tartarughe  ---  tartaruga
serpenti  ---  serpere
dipartimento di nettezza urbana  ---  dipartimento_nettezza_urbano
vettore  ---  vettore
cappellaccio  ---  cappellaccio
polvere fine  ---  polvere
teratogenicità  ---  teratogenicità
codice  ---  codice
fluorizzazione  ---  fluorizzazione
campionamento  ---  campionamento
benzina  ---  benzina
catione  ---  catione
indirizzario  ---  indirizzario
zoologia  ---  zoologia
organo  ---  organo
imenotteri  ---  imenotteri
registrazione di sostanze 

cimici  ---  cimice
colorimetria  ---  colorimetria
cianato  ---  cianato
vulcanizzazione  ---  vulcanizzazione
purina  ---  purina
norma  ---  norma
triturazione  ---  triturazione
isomero  ---  isomero
attinio  ---  attinio
agente mutageno  ---  agire_mutageno
olfattometria  ---  olfattometria
germoplasma  ---  germoplasma
interpolazione  ---  interpolazione
colore  ---  colorire
=== EXCLUDE  cloroetilene
silenziatore  ---  silenziatore
glossario  ---  glossario
carcinogeno  ---  carcinogeno
CD-ROM  ---  cd_rom
nitrosammine  ---  nitrosammine
straccio  ---  stracciare
sgrassaggio  ---  sgrassaggio
iceberg  ---  iceberg
addolcimento  ---  addolcimento
combustibile petrolio-alcool  ---  combustibile_petrolio_alcool
rigenerazione (combustibile nucleare)  ---  rigenerazione_combustibile_nucleare
liscivia  ---  lisciviare
=== EXCLUDE  disinchiostratura
ecozona  ---  ecozona
posidonia  ---  posidonia
ecomuseo  ---  ecomuseo
marchio di qualità ecologica  ---  marchiare_qualità_ecologico
===

slavina  ---  slavina
poriferi  ---  poriferi
inventario della merce  ---  inventariare_merce
bioetica  ---  bioetico
tracheofite  ---  tracheofite
UNCED  ---  unced
ursidi  ---  ursidi
bioreattore  ---  bioreattore
biosicurezza  ---  biosicurezza
pozzo  ---  pozzo
contabilità  ---  contabilità
Excluded:  57
Final topics:  5251 5251


In [28]:
# adding all the words and topics to model

# load the lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()
# load the nolemma list
with open(nolemma_list_file_excluded) as fp:
    topics_nolemma = fp.readlines()    
    

#try if the topic-lemma words exist in the model     
new_vectors = []
new_words = []
for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
#for i,topic in enumerate(topics):
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text) 
            
    topic = topic.strip()
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    #if len(topic_words)<2:
    #    continue
    
    
    tmp_vectors = []
    
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        vector = get_vector(word)
        
        try:
            tmp_vectors.append(model[word])
        except: 
            print('\nNOT in vocabulary: ', word, topic)
            vector = get_vector(word)
            if vector != []:
                tmp_vectors.append(vector)
                if word not in new_words:
                    print('append ', word)
                    new_vectors.append(vector)
                    new_words.append(word)   
            else:
                vector = get_vector(word_nolemma)
                if vector != []:
                    tmp_vectors.append(vector)
                    if word not in new_words:
                        print('append ', word)
                        new_vectors.append(vector)
                        new_words.append(word)  
                else:
                    raise Warning(f"XXX {word} {topic} {topic_nolemma}")
                    print("XXX", word, topic, topic_nolemma) 
    #if word not in new_words:
    #    new_vectors.append(list(np.average(tmp_vectors, axis=0)))
    #    new_words.append(word)        
    if len(tmp_vectors) > 1:
        if topic not in new_words:
   
            print('append ', topic)
            new_vectors.append(list(np.average(tmp_vectors, axis=0)))
            new_words.append(topic)
        else:
            print('doubled TOPIC: ', topic)    
    else:
        print('--- only one word, not a new topic>', topic)

append  sviluppare_locale
append  sviluppare_rurale
append  gestione_risorsa_idrico
append  grado_d_istruzione
--- only one word, not a new topic> istruzione
--- only one word, not a new topic> argine
append  diritto_comunitario
append  patrimonio_culturale
append  terreno_agricolo
append  paesaggio_coltivare
append  produzione_agricolo
append  sviluppare_economico
append  catena_alimentare
append  industriare_alimentare
append  qualità_alimento
append  riscaldamento_globale_atmosfera
append  effetto_serrare
append  sviluppare_industriale
--- only one word, not a new topic> mercato
append  accesso_territorio
append  accorpamento_parcellare
append  pianificazione_territoriale
append  architettura_paesaggio
append  pianificazione_paesaggio
append  usare_territorio
append  pianificazione_utilizzazione_territorio
append  ambiente_marinare
append  conservazione_natura
append  protezione_natura
append  coltura_biologico
append  pianificazione_spaziare_fisico
append  istruzione_elementare
app

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


--- only one word, not a new topic> investimento
append  problema_sociale
append  ambiente_fisico
append  ambiente_rurale
append  ambiente_terrestre
append  regolamentare_sanitario
append  sviluppare_sociale
--- only one word, not a new topic> giovane
append  contributo_finanziario
append  politico_territoriale
append  pianificazione_regionale
append  industriare_bevanda
append  produzione_industriale
append  gestione_foresta
append  acqua_dolce
append  navigazione_interno
append  gestione_energetico
append  rendimento_energetico
--- only one word, not a new topic> industriare
append  igiene_alimentare
append  produzione_raccolto
append  produzione_animale
append  credito_fiducia
append  inquinamento_suolare
append  inquinamento_acqua
append  tenere_agricolo
append  nuovo_comunità
--- only one word, not a new topic> economia
append  conquistare_territorio
append  immagazzinamento_derrata_alimentario
append  nutrizione_animale
append  produzione_forestale
append  pianificazione_agricolo

append  pianificazione_spaziare_aereo
append  area_terrestre
append  effetto_rumore
append  metodo_agricolo
append  crescita_popolazione
append  fertilità_suolare
append  politico_scientifico

NOT in vocabulary:  aridocoltura aridocoltura
append  aridocoltura
--- only one word, not a new topic> aridocoltura
append  dinamico_popolazione
append  direttivo_ce
append  bene_culturale
append  produrre_chimico_ambiente
append  disposizione_legale
append  normativo_regionale
append  politico_ambientale
append  foresta_naturale
append  industriare_cuoio
append  depurazione_suolare
append  miglioramento_suolare
append  geologia_marino
append  sistemare_protezione
append  processo_industriale
append  degradazione_inquinare
append  industriare_energetico
append  industriare_ambientale
append  prosciugamento_suolare
--- only one word, not a new topic> reddito
append  protezione_sito
append  sfruttamento_razionale_corso_d_acqua
--- only one word, not a new topic> vento
append  informazione_pubblico


append  pianto_acquatico
--- only one word, not a new topic> credito
--- only one word, not a new topic> finanziamento
append  industriare_minerario
append  politico_commerciale
append  autorità_regionale
--- only one word, not a new topic> carbonio
append  livellare_efficacia
append  sito_valore_storico
append  provvedimento_pubblicare
append  area_edificabile
append  effetto_tossico
append  carbonio_organico
append  rifiutare_agricolo
append  rifiutare_vegetale
append  bilanciare_ambientale
--- only one word, not a new topic> rilievo
append  politico_pescare_industriale
append  formare_gestione
append  acqua_balneazione
append  licenza_ambientale
append  società_industriale
append  gestione_patrimoniale
append  foresta_privato
append  attività_funerario
append  indicatore_culturale
append  riserva_forestale
append  economia_energetico
append  settore_primario
append  idraulico_agricolo
append  acqua_potabile
append  materiale_naturale
append  bozza_leggere
append  livellare_sviluppar

...

--- only one word, not a new topic> libertà
append  percentuale_riciclaggio
--- only one word, not a new topic> procarioti
append  previsione_pericolare
--- only one word, not a new topic> acidità
append  aroma_sostanza
append  prestito_biblioteca
append  sostanza_pericoloso_l_ambiente
--- only one word, not a new topic> aerosol
--- only one word, not a new topic> insetticida
--- only one word, not a new topic> ordinanza
append  acquisizione_prova
append  scavare_fossa
--- only one word, not a new topic> periziare
append  geologia_petrolifero
--- only one word, not a new topic> stress
append  residuare_filtrazione
append  immissione_polvere
append  raffinazione_carbone
append  eliminazione_carogna
append  raccolto_differenziato
append  cavare_tensione_sospendere

NOT in vocabulary:  parcare parcare_divertimento
append  parcare_divertimento
--- only one word, not a new topic> responsabilità
append  sostanza_tossico
--- only one word, not a new topic> furani

NOT in vocabulary:  alicicli

--- only one word, not a new topic> vernice
append  clorurare_polivinile
--- only one word, not a new topic> rivelatore
--- only one word, not a new topic> rischiare
--- only one word, not a new topic> rospo
--- only one word, not a new topic> pulitura
append  bilanciare_ecologico_politico_ambientale
append  udienza_ufficiale
append  contenitore_rifiuto
append  tessere_produrre
append  consenso_previo_informazione
--- only one word, not a new topic> tossicologia
--- only one word, not a new topic> reattore
append  combustibile_derivare_rifiuto
--- only one word, not a new topic> consultazione
append  paratoia_piano
--- only one word, not a new topic> armare
--- only one word, not a new topic> politico
append  organismo_bersagliare
append  trinità_principiare
--- only one word, not a new topic> fiore
--- only one word, not a new topic> geofisico

NOT in vocabulary:  teratogena sostanza_teratogena
append  teratogena
append  sostanza_teratogena
append  assorbimento_esposizione
--- only on

append  area_smistamento
--- only one word, not a new topic> newsgroup
--- only one word, not a new topic> sinergismo
--- only one word, not a new topic> clonazione
--- only one word, not a new topic> lineare
--- only one word, not a new topic> forno
--- only one word, not a new topic> risparmiare
append  licenza_cacciare
--- only one word, not a new topic> paleontologia
--- only one word, not a new topic> fumigazione
append  contenitore_rendere
append  sondaggio_d_opinione
--- only one word, not a new topic> parassitologia

NOT in vocabulary:  triazine triazine
append  triazine
--- only one word, not a new topic> triazine
--- only one word, not a new topic> puntare
--- only one word, not a new topic> collare
append  procedimento_preliminare
append  immissione_sonoro
--- only one word, not a new topic> avicoltura
--- only one word, not a new topic> nomenclatura
append  elemento_traccia
--- only one word, not a new topic> malformazione
append  depositare_ferraglia
--- only one word, not

--- only one word, not a new topic> forbicine

NOT in vocabulary:  bifenili bifenili_polibromurati

NOT in vocabulary:  polibromurati bifenili_polibromurati
append  polibromurati
append  bifenili_polibromurati

NOT in vocabulary:  bifenili bifenili_policlorurati

NOT in vocabulary:  policlorurati bifenili_policlorurati
append  bifenili_policlorurati
--- only one word, not a new topic> bibliografia
--- only one word, not a new topic> conservare

NOT in vocabulary:  spiaggiamento spiaggiamento
append  spiaggiamento
--- only one word, not a new topic> spiaggiamento
--- only one word, not a new topic> deprezzamento
--- only one word, not a new topic> odore
--- only one word, not a new topic> statistico
append  dato_geo_referenziare
--- only one word, not a new topic> ddt
--- only one word, not a new topic> enciclopedia
--- only one word, not a new topic> dendrocronologia
--- only one word, not a new topic> cimice
--- only one word, not a new topic> colorimetria

NOT in vocabulary:  cianato

--- only one word, not a new topic> bioreattore
--- only one word, not a new topic> biosicurezza
--- only one word, not a new topic> pozzo
--- only one word, not a new topic> contabilità


In [29]:
# add topics to model
model.add(new_words,new_vectors)

In [30]:
# save the model with new topics
model.save_word2vec_format(excluded_model_name, binary=True)
#reload the model  otherwise cannot find similarities fo newly added words
model = KeyedVectors.load_word2vec_format(excluded_model_name, binary=True)

In [31]:
len(model.index2word)

164954

In [32]:
# try the model
model.similar_by_vector('fertilizzare_chimico')

[('chimico', 0.93498295545578),
 ('produrre_chimico', 0.925504207611084),
 ('controllare_chimico', 0.9154489040374756),
 ('chimico_inorganico', 0.9143566489219666),
 ('decontaminazione_chimico', 0.9124979972839355),
 ('strutturare_chimico', 0.9124738574028015),
 ('inquinare_chimico', 0.9113664031028748),
 ('impiantire_chimico', 0.9076324701309204),
 ('precipitazione_chimico', 0.9065037369728088),
 ('contaminazione_chimico', 0.9059938788414001)]

In [33]:
# find most similar to topics to terms in the new model

# load the new lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

for i, topic in enumerate(topics):
    topic = topic.strip()
    print('\n-----\n', topic)
    try:
        print(i, model.similar_by_vector(topic))
    except:
        print(topic, 'NOT in model')


-----
 sviluppare_locale
0 [('strutturare_locale', 0.9615191221237183), ('locale', 0.9326653480529785), ('trafficare_locale', 0.9016720056533813), ('politico_locale', 0.8750431537628174), ('servizio_trasportare_locale', 0.8545322418212891), ('materiale_locale_costruzione', 0.8331085443496704), ('autorità_locale', 0.8102292418479919), ('finanza_locale', 0.8077927827835083), ('turismo_locale', 0.7974005937576294), ('utilizzazione_risorsa_locale', 0.7875776886940002)]

-----
 sviluppare_rurale
1 [('rurale', 0.9462684392929077), ('pianificazione_rurale', 0.9230279326438904), ('insediamento_rurale', 0.9108104705810547), ('alloggiare_rurale', 0.9107690453529358), ('architettura_rurale', 0.9092286825180054), ('popolazione_rurale', 0.9053616523742676), ('ambiente_rurale', 0.8922696113586426), ('diritto_rurale', 0.8530783653259277), ('habitat_rurale', 0.8103694319725037), ('area_rurale', 0.7735484838485718)]

-----
 gestione_risorsa_idrico
2 [('risorsa_idrico', 0.9667526483535767), ('conservaz

24 [('classificazione_usare_territorio', 0.9443771839141846), ('usare_paesaggio', 0.8494497537612915), ('usare', 0.8458645343780518), ('usare_sostenibile', 0.839145302772522), ('usare_liberare', 0.817261815071106), ('leggere_usare_fertilizzare', 0.8159644603729248), ('utilizzare', 0.793302059173584), ('combustibile_usare_marittimo', 0.792656660079956), ('usare_coercitivo', 0.7880550622940063), ('usare_eccessivo_fertilizzare', 0.7848818302154541)]

-----
 pianificazione_utilizzazione_territorio
25 [('pianificazione_territoriale', 0.8474785685539246), ('conservazione_territorio', 0.797744631767273), ('sfruttamento_razionale_territorio', 0.7782918214797974), ('pianificazione_ambientale', 0.771072506904602), ('territorio', 0.7696704268455505), ('intervento_territorio', 0.766488790512085), ('pianificazione', 0.7562726736068726), ('pianificazione_industriale', 0.7555923461914062), ('pianificazione_regionale', 0.7547089457511902), ('amministrazione_territoriale', 0.747521162033081)]

-----
 a

48 [('litoraneo', 0.842400312423706), ('ecosistema_marinare', 0.7948247194290161), ('ecosistema', 0.791283130645752), ('ecosistema_subtropicale', 0.7885627746582031), ('ecosistema_territorio_semiaridi', 0.7801253795623779), ('degradare_ecosistema', 0.7709169387817383), ('ecosistema_altopiano', 0.7680280804634094), ('ecosistema_acquatico', 0.7651358842849731), ('ambiente_costiero', 0.753563642501831), ('ecosistema_estuario', 0.7470454573631287)]

-----
 ecosistema_acqua_dolce
49 [('inquinamento_acqua_dolce', 0.9647353291511536), ('acqua_dolce', 0.9630264043807983), ('monitoraggio_acqua_dolce', 0.957055389881134), ('conservazione_acqua_dolce', 0.955240786075592), ('organismo_acqua_dolce', 0.9528653025627136), ('degradare_acqua_dolce', 0.951387345790863), ('biologia_acqua_dolce', 0.9434700012207031), ('risorsa_acqua_dolce', 0.9358220100402832), ('acqua_dolce_idoneo_balneazione', 0.8936202526092529), ('acqua_salato', 0.8722274303436279)]

-----
 ecosistema_acquatico
50 [('ambiente_acquatic

74 [('conservazione_natura', 0.961870551109314), ('programmare_conservazione_natura', 0.9368757009506226), ('legislazione_conservazione_natura', 0.9364538192749023), ('politico_conservazione_natura', 0.9093503952026367), ('natura', 0.8702348470687866), ('protezione_natura', 0.8504340052604675), ('ritornare_natura', 0.803054690361023), ('diritto_natura', 0.8005867004394531), ('istituzione_pubblico_natura_amministrativo', 0.7900856733322144), ('diritto_indipendenza_natura', 0.7812523245811462)]

-----
 industriare_lavorazione_alimento
75 [('commerciare_alimento', 0.8435471057891846), ('contaminazione_alimento', 0.8082776069641113), ('conservazione_alimento', 0.8051362633705139), ('alimento', 0.8035030364990234), ('industriare_lavorazione_minerale', 0.7911249399185181), ('irradiazione_alimento', 0.7743560671806335), ('infestazione_alimento', 0.7599726915359497), ('industriare_lavorazione_rifiuto', 0.7491422891616821), ('qualità_alimento', 0.746841549873352), ('industriare_imballaggio', 0.

101 [('maggengo', 0.5103673934936523), ('roveja', 0.4525211751461029), ('trebbiatura', 0.44695574045181274), ('granturco', 0.4439590573310852), ('semente', 0.4420870244503021), ('aiola', 0.44183990359306335), ('aratura', 0.43890446424484253), ('zappa', 0.43840542435646057), ('pianto_foraggera', 0.4369344413280487), ('aratro', 0.43607187271118164)]

-----
 istruzione_superiora
102 [('superiora', 0.8632153272628784), ('istruzione', 0.7603691816329956), ('istruzione_secondario', 0.7076903581619263), ('istruzione_amministrativo', 0.6971356868743896), ('istruzione_elementare', 0.6941083073616028), ('istruzione_tecnico', 0.6481439471244812), ('politico_istruzione', 0.6185848712921143), ('superiore', 0.5637590885162354), ('suora', 0.5528262853622437), ('organizzazione_scolastico', 0.53511643409729)]

-----
 regime_fondiario
103 [('regime', 0.8818515539169312), ('regime_conduzione', 0.8298125267028809), ('regime_fiscale', 0.8002430200576782), ('fondiario', 0.7681200504302979), ('regime_effluss

...

5151 [('legislazione_ambientale', 0.909610390663147), ('normativo', 0.8718035817146301), ('ambientale', 0.8519715070724487), ('normativo_regionale', 0.8471269607543945), ('terminologia_ambientale', 0.845694899559021), ('processare_legislativo_ambientale', 0.8381164073944092), ('informazione_ambientale', 0.8316011428833008), ('legislazione_ambientale_agricolo', 0.828458309173584), ('formazione_ambientale', 0.8273178935050964), ('informatico_ambientale', 0.8218222856521606)]

-----
 spaziare_ambientale
5152 [('sviluppare_ambientale', 0.8559855222702026), ('programmare_ambientale', 0.8219932317733765), ('ricercare_ambientale', 0.8138009309768677), ('bilanciare_ambientale', 0.8123980760574341), ('incentivare_ambientale', 0.8061692714691162), ('impattare_ambientale_industriare', 0.8061223030090332), ('rapportare_impattare_ambientale', 0.8022336959838867), ('impattare_ambientale', 0.8007123470306396), ('studiare_impattare_ambientale', 0.8002438545227051), ('controllare_ambientale', 0.7996087

5173 [('crosta', 0.908981204032898), ('crosta_terrestre', 0.841544508934021), ('bilanciare_massa_crosta_ghiacciato', 0.8358644843101501), ('ghiacciato', 0.7342760562896729), ('calotta_glaciale', 0.6388742327690125), ('ghiaccio', 0.619694709777832), ('ghiacciare', 0.6195815801620483), ('ghiacciata', 0.6058506369590759), ('crostoni', 0.5988835692405701), ('ghiaccia', 0.581755518913269)]

-----
 bilanciare_massa_crosta_ghiacciato
5174 [('crosta_ghiacciato', 0.8358645439147949), ('ricreazione_massa', 0.7878294587135315), ('massa', 0.7731083631515503), ('crosta', 0.7663553357124329), ('crosta_terrestre', 0.7623479962348938), ('trasportare_massa_fisico', 0.7491041421890259), ('ghiacciato', 0.6030550003051758), ('perdita_ghiacciare', 0.600026547908783), ('miscelare_gassoso', 0.5992894172668457), ('sfruttamento_fondere_marinare', 0.5968366861343384)]

-----
 degradare_suolare
5175 [('suolare_contaminare', 0.9629275798797607), ('suolare', 0.9496582746505737), ('inquinare_suolare', 0.94839704036

5198 [('direttivo_tutelare_acqua', 0.9310999512672424), ('direttivo_acqua_balneazione', 0.9252661466598511), ('direttivo_qualità_acqua', 0.8868753910064697), ('sistemare_distribuzione_acqua', 0.8807024955749512), ('livellare_acqua', 0.8704569935798645), ('riservare_acqua', 0.8688656687736511), ('acqua_interno', 0.8683617115020752), ('statistico_acqua_rifiutare', 0.8663783073425293), ('legislazione_acqua_rifiutare', 0.86557537317276), ('statistico_acqua', 0.8648656606674194)]

-----
 rifiutare_domestico
5199 [('domestico', 0.8287580013275146), ('apparecchiatura_domestico', 0.7725125551223755), ('rifiutare_alimentare', 0.7515144348144531), ('discaricare_rifiuto_domestico', 0.7369544506072998), ('rifiutare_industriale', 0.7367208003997803), ('rifiutare_consumere', 0.7363278865814209), ('combustibile_domestico', 0.723369300365448), ('rifiutare_medicare', 0.7219028472900391), ('produrre_chimico_usare_domestico', 0.7200192213058472), ('rifiutare', 0.719875693321228)]

-----
 fumare_ambiente_

5224 [('idroponica', 0.9193315505981445), ('coltivazione', 0.8326216340065002), ('idroponico', 0.7828044891357422), ('coltivazione_substrato', 0.7724485993385315), ('idroponici', 0.7471091747283936), ('coltivazione_commerciale', 0.7073559165000916), ('coltura_vivaio', 0.6906381845474243), ('infestare_coltura_agricolo', 0.6841791868209839), ('coltura', 0.6810441017150879), ('coltura_industriale', 0.6796305179595947)]

-----
 infrasuono
5225 [('infrasuoni', 0.7047046422958374), ('ultrasuono', 0.6049039363861084), ('spettrogramma', 0.4877593219280243), ('elettromagnetico', 0.4838404357433319), ('misurare_rumore', 0.462824285030365), ('binaurale', 0.46242856979370117), ('spettro_rumore', 0.46161502599716187), ('impostare_emissione_rumore', 0.46125054359436035), ('ultrasuonoterapia', 0.45520633459091187), ('rumore_intermittente', 0.4520081579685211)]

-----
 riscaldamento_domestico_fornire_impianto_industriale_situare_vicin
5226 [('impiantire_industriale', 0.6897866725921631), ('impiantire_

5249 [('pozzo_acqua', 0.8552985191345215), ('gotto', 0.6089306473731995), ('artesiano', 0.597754955291748), ('pozzi', 0.5877146124839783), ('sistemare_trattamento_acqua_fognare', 0.5773071646690369), ('cunicolo', 0.5771362781524658), ('pozzetto', 0.5763437747955322), ('smaltimento_acqua_fognare', 0.5763345956802368), ('acqua_sotterraneo', 0.572941780090332), ('cavo_sotterraneo', 0.5714105367660522)]

-----
 contabilità
5250 [('sistemare_contabilità', 0.8486005067825317), ('contabilita', 0.8265488147735596), ('contabilità_ambientale', 0.8215408325195312), ('gestione_contabile', 0.7758894562721252), ('contabile', 0.7448537349700928), ('contabilità_ecologico', 0.7341648936271667), ('contabilizzazione', 0.6906017661094666), ('contabilit', 0.6556664109230042), ('contabili', 0.653057336807251), ('gestione_finanziario', 0.6073857545852661)]
